# Catchment Boundary Derivation — ausvic (FloodHubMaribyrnong)

**Prerequisite:** Run `0a-derive_gauge_config_ausvic.ipynb` first — it saves
`gauges_ausvic.json` to Google Drive which this notebook loads.

Derives upstream catchment boundary polygons for 10 Maribyrnong River gauging
stations using **HydroBASINS Level 12** hosted in Google Earth Engine.

**Method**
1. Load `GAUGES` from `gauges_ausvic.json` (output of 0a)
2. Find the HydroBASINS Level-12 cell containing each gauge point
3. Iteratively trace upstream via `NEXT_DOWN` links
4. Union all upstream cells into a single catchment polygon per gauge
5. Save combined GeoJSON + ESRI Shapefile to Google Drive
6. Visualise catchments (static map + interactive map)

**Output** (saved to `My Drive/caravan_maribyrnong_gee/shapefiles/`)
- `ausvic_basin_shapes.geojson` — combined FeatureCollection
- `ausvic_basin_shapes.shp/.shx/.dbf/.prj/.cpg` — Caravan-format shapefile
- `ausvic_catchments_map.png` — static map image

**Next step:** Upload the shapefile to GEE as an asset named
`projects/floodhubmaribyrnong/assets/ausvic_basin_shapes`
for use in `Caravan_part1_Earth_Engine_ausvic.ipynb`.

In [ ]:
# Install dependencies (earthengine-api is pre-installed in Colab)
!pip install -q geopandas folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='floodhubmaribyrnong')

In [ ]:
import json
from pathlib import Path

# HydroBASINS Level 12 — hosted in GEE
GEE_BASINS = 'WWF/HydroSHEDS/v1/Basins/hybas_12'

# ── Paths ─────────────────────────────────────────────────────────────────────
# Google Drive output directory (default for Colab)
OUT_DIR = Path('/content/drive/MyDrive/caravan_maribyrnong_gee/shapefiles')

# Local testing override — set this to your local GeoJSON path to run the
# visualisation cells without Colab or Google Drive. Leave as None for Drive.
LOCAL_GEOJSON = None
# LOCAL_GEOJSON = Path(r'C:\Users\leela\FloodHubMaribyrnong\caravan_maribyrnong\shapefiles\ausvic\ausvic_basin_shapes.geojson')

if LOCAL_GEOJSON is not None:
    LOCAL_GEOJSON = Path(LOCAL_GEOJSON)
    OUT_DIR = LOCAL_GEOJSON.parent
    print(f'Local mode — GeoJSON : {LOCAL_GEOJSON}')
    print(f'Output dir           : {OUT_DIR}')
else:
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    print(f'Drive mode — output dir: {OUT_DIR}')

In [ ]:
# ── Load GAUGES ───────────────────────────────────────────────────────────────
# Primary  : JSON saved by 0a-derive_gauge_config_ausvic.ipynb to Google Drive.
# Fallback : gauges_config.py from the repo root (local testing without Colab).

GAUGES_JSON = Path('/content/drive/MyDrive/caravan_maribyrnong_gee/gauges_ausvic.json')

if GAUGES_JSON.exists():
    with open(GAUGES_JSON) as f:
        GAUGES = json.load(f)
    print(f'Loaded {len(GAUGES)} gauges from Drive JSON')
else:
    # Local fallback — import from gauges_config.py in the repo root
    import sys
    repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
    sys.path.insert(0, str(repo_root))
    try:
        import importlib, gauges_config
        importlib.reload(gauges_config)   # picks up any edits without kernel restart
        _raw = gauges_config.GAUGES
        # Serialise date objects to ISO strings so GAUGES is JSON-compatible
        GAUGES = json.loads(json.dumps(
            [{k: (v.isoformat() if hasattr(v, 'isoformat') else v) for k, v in g.items()}
             for g in _raw]
        ))
        print(f'Loaded {len(GAUGES)} gauges from gauges_config.py (local fallback)')
    except ImportError:
        raise FileNotFoundError(
            f'{GAUGES_JSON} not found and gauges_config.py could not be imported.\n'
            'Either run 0a-derive_gauge_config_ausvic.ipynb first, '
            'or run this notebook from the repo root.'
        )

for g in GAUGES:
    print(f"  {g['gauge_id']:20s}  {g['name']}")

In [ ]:
def trace_upstream(lat, lon, gauge_id):
    """
    Collect every HydroBASINS Level-12 cell upstream of (lat, lon),
    union them into one polygon, and return a GeoJSON Feature.
    """
    basins = ee.FeatureCollection(GEE_BASINS)
    point  = ee.Geometry.Point([lon, lat])

    # 1. Find the outlet basin cell
    print(f'  Finding outlet basin at ({lat}, {lon}) ...')
    outlet      = basins.filterBounds(point).first()
    outlet_info = outlet.getInfo()

    if outlet_info is None:
        raise RuntimeError(f'No HydroBASINS basin found at ({lat}, {lon})')

    props     = outlet_info['properties']
    outlet_id = props['HYBAS_ID']
    up_area   = props['UP_AREA']
    print(f'  Outlet HYBAS_ID : {outlet_id}')
    print(f'  Outlet UP_AREA  : {up_area:.1f} km2')

    # 2. Iteratively trace upstream via NEXT_DOWN links
    print('  Tracing upstream basins ...')
    all_ids  = {outlet_id}
    frontier = {outlet_id}
    iteration = 0

    while frontier:
        iteration += 1
        parents    = basins.filter(ee.Filter.inList('NEXT_DOWN', list(frontier)))
        parent_ids = parents.aggregate_array('HYBAS_ID').getInfo()
        new_ids    = set(parent_ids) - all_ids

        print(f'    Iteration {iteration}: {len(new_ids)} new basin(s) '
              f'(running total: {len(all_ids) + len(new_ids)})')

        if not new_ids:
            break

        all_ids.update(new_ids)
        frontier = new_ids

    print(f'  Total basins in catchment: {len(all_ids)}')

    # 3. Union all upstream polygons (30 m tolerance)
    print('  Unioning polygons ...')
    upstream_fc = basins.filter(ee.Filter.inList('HYBAS_ID', list(all_ids)))
    merged      = upstream_fc.union(maxError=30).first()
    merged_info = merged.getInfo()

    return {
        'type': 'Feature',
        'properties': {
            'gauge_id':    gauge_id,
            'up_area_km2': up_area,
        },
        'geometry': merged_info['geometry'],
    }

In [ ]:
# ── Run upstream trace for all 10 gauges ─────────────────────────────────────
all_features = []

for gauge in GAUGES:
    gid = gauge['gauge_id']
    print(f"
{'=' * 60}")
    print(f"Gauge: {gauge['name']} ({gid})")
    print(f"{'=' * 60}")

    known_area = gauge.get('area_km2')
    if known_area:
        print(f'  Known catchment area: {known_area} km2')

    try:
        feature = trace_upstream(gauge['lat'], gauge['lon'], gid)
    except Exception as exc:
        print(f'  ERROR: {exc}')
        continue

    hydrobasins_area = feature['properties']['up_area_km2']
    if known_area:
        diff_pct = abs(hydrobasins_area - known_area) / known_area * 100
        print(f'  Area check: HydroBASINS={hydrobasins_area:.1f} km2  '
              f'known={known_area} km2  diff={diff_pct:.1f}%')

    all_features.append(feature)

print(f'
Processed {len(all_features)} / {len(GAUGES)} gauges successfully')

In [ ]:
# ── Save GeoJSON ──────────────────────────────────────────────────────────────
# Strip up_area_km2 — Caravan standard requires gauge_id only

clean_features = [
    {
        'type': 'Feature',
        'properties': {'gauge_id': f['properties']['gauge_id']},
        'geometry': f['geometry'],
    }
    for f in all_features
]

fc = {'type': 'FeatureCollection', 'features': clean_features}

geojson_path = OUT_DIR / 'ausvic_basin_shapes.geojson'
geojson_path.write_text(json.dumps(fc, indent=2))

print(f'GeoJSON saved: {geojson_path}')
print(f'  Features : {len(clean_features)}')
print(f'  IDs      : {[f["properties"]["gauge_id"] for f in clean_features]}')

In [ ]:
# ── Save ESRI Shapefile ───────────────────────────────────────────────────────
# Caravan requires a SINGLE combined shapefile with gauge_id column only.

import geopandas as gpd

gdf = gpd.read_file(str(geojson_path))
gdf = gdf[['gauge_id', 'geometry']]   # Caravan: gauge_id column only

shp_path = OUT_DIR / 'ausvic_basin_shapes.shp'
gdf.to_file(str(shp_path))            # writes .shp/.shx/.dbf/.prj/.cpg

print(f'Shapefile saved: {shp_path}')
print(gdf[['gauge_id']].to_string())

## Visualise Catchment Boundaries

Two views of the 10 catchment polygons:
- **Static map** — saved as PNG to Google Drive
- **Interactive map** — explore in-notebook with zoom and click popups

In [ ]:
# ── Static map (matplotlib) ───────────────────────────────────────────────────
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Load GeoJSON if gdf is not already in memory
try:
    gdf
except NameError:
    _path = LOCAL_GEOJSON if LOCAL_GEOJSON is not None else (OUT_DIR / 'ausvic_basin_shapes.geojson')
    gdf = gpd.read_file(str(_path))
    print(f'Loaded GeoJSON from {_path}')

# 10 visually distinct colours
COLORS = [
    '#e6194b', '#3cb44b', '#4363d8', '#f58231', '#911eb4',
    '#42d4f4', '#f032e6', '#bfef45', '#fabed4', '#469990',
]

# Build gauge_id -> colour mapping (same order as GAUGES list)
gauge_color = {g['gauge_id']: COLORS[i] for i, g in enumerate(GAUGES)}

fig, ax = plt.subplots(figsize=(10, 12))

# Plot catchment polygons
for _, row in gdf.iterrows():
    color = gauge_color.get(row['gauge_id'], '#aaaaaa')
    gpd.GeoDataFrame([row], crs=gdf.crs).plot(
        ax=ax, color=color, alpha=0.35, edgecolor='black', linewidth=0.8
    )

# Plot gauge point markers and labels
gauge_lookup = {g['gauge_id']: g for g in GAUGES}
for gid, color in gauge_color.items():
    g = gauge_lookup[gid]
    ax.plot(
        g['lon'], g['lat'], 'o',
        color=color, markersize=9,
        markeredgecolor='black', markeredgewidth=0.8, zorder=5
    )
    short_label = gid.replace('ausvic_', '')
    ax.annotate(
        short_label,
        xy=(g['lon'], g['lat']),
        xytext=(5, 4), textcoords='offset points',
        fontsize=7.5, fontweight='bold', zorder=6,
        bbox=dict(boxstyle='round,pad=0.15', facecolor='white', alpha=0.75, edgecolor='none')
    )

# Legend — gauge_id + name + area
legend_patches = [
    mpatches.Patch(
        facecolor=gauge_color[g['gauge_id']], alpha=0.6, edgecolor='black', linewidth=0.5,
        label=f"{g['gauge_id']}  {g['name'].split(' at ')[-1]}  ({g['area_km2']:.0f} km\u00b2)"
    )
    for g in GAUGES
]
ax.legend(
    handles=legend_patches, loc='lower left', fontsize=7,
    title='Gauge ID  |  Location  |  Catchment area',
    title_fontsize=7.5, framealpha=0.9
)

ax.set_title(
    'Maribyrnong Catchment — 10 Gauging Stations\nausvic Caravan Submission',
    fontsize=13, pad=12
)
ax.set_xlabel('Longitude (deg E)')
ax.set_ylabel('Latitude (deg N)')
ax.set_aspect('equal')
plt.tight_layout()

map_png = OUT_DIR / 'ausvic_catchments_map.png'
plt.savefig(str(map_png), dpi=150, bbox_inches='tight')
plt.show()
print(f'Static map saved: {map_png}')

In [ ]:
# ── Interactive map (folium) ──────────────────────────────────────────────────
import subprocess, sys
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'folium'], check=False)
import folium

# Load GeoJSON if clean_features not already in memory
try:
    clean_features
except NameError:
    _path = LOCAL_GEOJSON if LOCAL_GEOJSON is not None else (OUT_DIR / 'ausvic_basin_shapes.geojson')
    with open(_path) as f:
        fc = json.load(f)
    clean_features = fc['features']
    print(f'Loaded GeoJSON from {_path}')

if 'COLORS' not in dir():
    COLORS = [
        '#e6194b', '#3cb44b', '#4363d8', '#f58231', '#911eb4',
        '#42d4f4', '#f032e6', '#bfef45', '#fabed4', '#469990',
    ]

center_lat = sum(g['lat'] for g in GAUGES) / len(GAUGES)
center_lon = sum(g['lon'] for g in GAUGES) / len(GAUGES)

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='CartoDB positron'
)

gauge_lookup = {g['gauge_id']: g for g in GAUGES}

for i, feat in enumerate(clean_features):
    gid   = feat['properties']['gauge_id']
    g     = gauge_lookup.get(gid)
    if g is None:
        continue
    color = COLORS[i % len(COLORS)]

    folium.GeoJson(
        feat,
        name=gid,
        style_function=lambda x, c=color: {
            'fillColor': c,
            'color':     'black',
            'weight':    1.2,
            'fillOpacity': 0.3,
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['gauge_id'],
            aliases=['Gauge:'],
            style='font-size:12px;'
        )
    ).add_to(m)

    folium.CircleMarker(
        location=[g['lat'], g['lon']],
        radius=8,
        color='black',
        weight=1.5,
        fill=True,
        fill_color=color,
        fill_opacity=1.0,
        popup=folium.Popup(
            f"<b>{gid}</b><br>"
            f"{g['name']}<br>"
            f"Area: {g['area_km2']} km\u00b2<br>"
            f"Lat: {g['lat']:.5f}, Lon: {g['lon']:.5f}",
            max_width=220
        ),
        tooltip=f"{gid} — {g['name']}"
    ).add_to(m)

    folium.Marker(
        location=[g['lat'], g['lon']],
        icon=folium.DivIcon(
            html=(
                f'<div style="'
                f'font-size:9px;font-weight:bold;'
                f'background:white;padding:1px 4px;'
                f'border-radius:3px;opacity:0.85;'
                f'white-space:nowrap;margin-left:10px;">'
                f'{gid.replace("ausvic_", "")}'
                f'</div>'
            ),
            icon_size=(80, 18),
            icon_anchor=(0, 9),
        )
    ).add_to(m)

folium.LayerControl().add_to(m)
m

## Next Step — Upload Shapefile to GEE

1. Download the shapefile from Google Drive:
   `My Drive/caravan_maribyrnong_gee/shapefiles/`
   (download all 5 files: `.shp`, `.shx`, `.dbf`, `.prj`, `.cpg`)

2. Go to [code.earthengine.google.com](https://code.earthengine.google.com/)

3. **Assets** tab → **NEW** → **Shape files**

4. Upload all 5 files together, name the asset:
   ```
   ausvic_basin_shapes
   ```
   Full path will be: `projects/floodhubmaribyrnong/assets/ausvic_basin_shapes`

5. Wait for ingestion to complete (Tasks tab), then run
   `Caravan_part1_Earth_Engine_ausvic.ipynb`